# Содержание:

    1) Метод word2vec с классификаторами:
            1) Ridge Classifier
            2) Perceptron
            3) Passive Agressive
            4) BeurnolliNB
            5) MultinomialNB
            6) KNeighbors
            7) RandomForest
            8) NearestCentroid
            9) SGDClassifier
            10) LinearSVC
    2) Визуализация


In [2]:
%matplotlib inline
import os
from lxml import etree
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Olivier Grisel <olivier.grisel@ensta.org>
#         Mathieu Blondel <mathieu@mblondel.org>
#         Lars Buitinck
# License: BSD 3 clause

from __future__ import print_function

import logging
import numpy as np
from optparse import OptionParser
import sys
from time import time
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import pymorphy2
import gensim
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict

# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

In [3]:
class DocIterator(object):
    def __init__(self, doc_list, labels_list):
       self.labels_list = labels_list
       self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield gensim.models.doc2vec.LabeledSentence(words=doc.split(),tags=[self.labels_list[idx]])

## Загружаю данные:

In [3]:
sphere_6 = defaultdict(int)
genre_fi_7 = defaultdict(int)
type_8 = defaultdict(int)
topic_9 = defaultdict(int)

i = 0
f = open('./out.csv', 'r')
for line in f:
    i += 1
    attrs = line.split(';')
    sphere_6[attrs[6]] += 1
    genre_fi_7[attrs[7]] += 1
    type_8[attrs[8]] += 1
    topic_9[attrs[9]] += 1

In [82]:
# type_8

In [80]:
# cl.keys()

In [4]:
from Levenshtein import distance

In [5]:
f = open('./out.csv', 'r')
file_hash_all = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    file_name = os.path.basename(path)
    dir_name = os.path.dirname(path)
    full_dir = os.path.join('/Users/Valeriya/Desktop/CoursePaper/corpus', dir_name)
#     full_dir = os.path.join('./corpus', dir_name)
    files = os.listdir(full_dir)
    if file_name in files:
        file_hash_all[file_name] = class_name
    elif file_name + '.xml' in files:
        file_hash_all[file_name + '.xml'] = class_name
    elif file_name + '.xhtml' in files:
        file_hash_all[file_name + '.xhtml'] = class_name
    else:
        edit = 1000
        true_name = ''
        for file_in_dir in files:
            dist = distance(file_name, file_in_dir)
            if dist < edit:
                edit = dist
                true_name = file_in_dir
        file_hash_all[true_name] = class_name

In [6]:
len(file_hash_all)

14382

In [134]:
f = open('./out.csv', 'r')
# cl = {'отчет':0,'поздравление':0, 'интервью':0, 'репортаж':0, 'хроника':0, 'объявление':0, 'комментарий':0, 'совет':0, 'анонс':0, 'заметка':0, 'информационное сообщение':0, 'статья':0}
cl = {'статья':0, 'интервью':0, 'комментарий':0}
file_hash = {}
for file_name, class_name in file_hash_all.items():
    if class_name in cl:
        cl[class_name] +=1
        file_hash[file_name] = class_name

In [135]:
len(file_hash)

4011

In [25]:
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    pos = p.tag.POS
    if pos:
        if 'ADJ' in pos:
            pos = 'ADJ'
    else:
        pos = ''
    return '{}_{}'.format(
        p.normal_form,
        pos
    )

2017-05-17 22:56:44,592 INFO Loading dictionaries from /Users/Valeriya/anaconda/lib/python3.5/site-packages/pymorphy2_dicts/data
2017-05-17 22:56:46,046 INFO format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [26]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text, open_name

In [27]:
# sent = []
# for tree, class_text, open_name in read_txt('./corpus'):
#     all_p = tree.xpath('//p')
#     joined = '\n'.join([p.text for p in all_p if p.text is not None])
#     sent.append((joined, class_text))

In [288]:
# words = sum([len(text[0].split()) for text in sent])
# words

In [27]:
texts = []
for tree, class_text, open_name in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
# for tree, class_text, open_name in read_txt('./corpus'):
    body = tree.xpath('//body')
    text = '\n'.join([elem.text for elem in body[0] if elem.text])
#     if class_text != 'поздравление':
#         class_text = 'другие'
    if text:
        texts.append((text, class_text, open_name))
    else:
        print(open_name)

/Users/Valeriya/Desktop/CoursePaper/corpus/rossija/nesnyat/south/sochi/451403.xhtml
/Users/Valeriya/Desktop/CoursePaper/corpus/rossija/nesnyat/south/sochi/451404.xhtml


In [30]:
# print(texts[0])

In [291]:
# words = sum([len(text[0].split()) for text in sent])
# words

In [28]:
len_texts = len(texts)

In [29]:
len_texts

9846

In [14]:
# m = 'ruwikiruscorpora_0_300_20.bin.gz'
# model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)

In [30]:
only_texts = [text[0] for text in texts]
classes = [text[1] for text in texts]

In [129]:
docLabels = [str(i) for i in range(len_texts)]

it = DocIterator(only_texts, docLabels)

model = gensim.models.Doc2Vec(window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(it)
for epoch in range(10):
    model.train(it)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca
    model.train(it)

2017-05-18 16:35:24,344 INFO collecting all words and their counts
2017-05-18 16:35:24,346 INFO PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-05-18 16:35:27,347 INFO collected 532997 word types and 9846 unique tags from a corpus of 9846 examples and 5330992 words
2017-05-18 16:35:27,348 INFO Loading a fresh vocabulary
2017-05-18 16:35:28,426 INFO min_count=5 retains 91016 unique words (17% of original 532997, drops 441981)
2017-05-18 16:35:28,427 INFO min_count=5 leaves 4684257 word corpus (87% of original 5330992, drops 646735)
2017-05-18 16:35:28,746 INFO deleting the raw counts dictionary of 532997 items
2017-05-18 16:35:28,806 INFO sample=0.001 downsamples 21 most-common words
2017-05-18 16:35:28,807 INFO downsampling leaves estimated 4150313 word corpus (88.6% of prior 4684257)
2017-05-18 16:35:28,808 INFO estimated required memory for 91016 words and 100 dimensions: 124228400 bytes
2017-05-18 16:35:29,222 INFO resetting layer weights
2017-05-18 16:35

In [16]:
len(model.docvecs)

4592

In [122]:
features = {}
# with open('formal_features_3.tsv', 'r') as f:
with open('formal_features_3.tsv', 'r') as f:
    for line in f:
        items = line.rstrip().split('\t')
        path = items[0]
        feats = items[1::]
        features[path] = feats

In [123]:
data = []
for i, (text, class_name, open_name) in enumerate(texts):
    text_vectors = []
    words = text.split()
    doc2vec_vector = list(model.docvecs[i])
    open_name = open_name.replace('/Users/Valeriya/Desktop/CoursePaper/', './')
    formal_features = features[open_name][::]
    n_excl = 0
    for item in text[0]:
        if item == '!':
            n_excl+=1

    n_ques = 0
    for item in text[0]:
        if item == '?':
            n_ques+=1
    formal_features.append(n_excl)
    formal_features.append(n_ques)
    
    formal_features = [float(x) for x in formal_features]
    formal_features = formal_features[:2:] + [x/formal_features[0] for x in formal_features[2::]]
    
    data.append(doc2vec_vector+formal_features)


# data = pd.DataFrame(texts_norm)
# data['class'] = classes

In [35]:
len(data)

9846

In [124]:
X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size=0.3)

In [116]:
min_max_scaler = MinMaxScaler()
X_train_norm = min_max_scaler.fit_transform(X_train)
X_test_norm = min_max_scaler.fit_transform(X_test)

## Классификаторы

In [37]:
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    if clf == MultinomialNB(alpha=.01):
        clf.fit(X_train, y_train)
    else:
        clf.fit(X_train_norm, y_train)
#     train_time = time() - t0
#     print("train time: %0.3fs" % train_time)

    t0 = time
    if clf == MultinomialNB(alpha=.01):
        pred = clf.predict(X_test)
    else:
        pred = clf.predict(X_test_norm)
        
#     test_time = time() - t0
#     print("test time:  %0.3fs" % test_time)

    score = metrics.accuracy_score(y_test, pred)
    print("accuracy:   %0.3f" % score)

    print()
    clf_descr = str(clf).split('(')[0]
    if "penalty" in vars(clf):
        return clf_descr+' '+str(clf.penalty), score
    else:
        return clf_descr, score

results = []
for clf, name in (
        (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
        (Perceptron(n_iter=50), "Perceptron"),
        (PassiveAggressiveClassifier(n_iter=50), "Passive-Aggressive"),
        (KNeighborsClassifier(n_neighbors=10), "kNN"),
        (RandomForestClassifier(n_estimators=100), "Random forest")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))

for penalty in ["l2", "l1"]:
    print('=' * 80)
    print("%s penalty" % penalty.upper())
    # Train Liblinear model
    results.append(benchmark(LinearSVC(loss='l2', penalty=penalty,
                                            dual=False, tol=1e-3)))

    # Train SGD model
    results.append(benchmark(SGDClassifier(alpha=.0001, n_iter=50,
                                           penalty=penalty)))

# Train SGD with Elastic Net penalty
print('=' * 80)
print("Elastic-Net penalty")
results.append(benchmark(SGDClassifier(alpha=.0001, n_iter=50,
                                       penalty="elasticnet")))

# Train NearestCentroid without threshold
print('=' * 80)
print("NearestCentroid (aka Rocchio classifier)")
results.append(benchmark(NearestCentroid()))

# Train Logistic Regression
print('=' * 80)
print("Logistic Regression")
results.append(benchmark(LogisticRegression()))

# Train sparse Naive Bayes classifiers
print('=' * 80)
print("Naive Bayes")
results.append(benchmark(MultinomialNB(alpha=.01)))
results.append(benchmark(BernoulliNB(alpha=.01)))

print('=' * 80)
print("LinearSVC with L1-based feature selection")
# The smaller C, the stronger the regularization.
# The more regularization, the more sparsity.
results.append(benchmark(Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC( penalty="l1", dual=False, tol=1e-3))),
  ('classification', LinearSVC())
])))

# make some plots

indices = np.arange(len(results))

results = [[x[i] for x in results] for i in range(2)]

Ridge Classifier
________________________________________________________________________________
Training: 
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='lsqr',
        tol=0.01)
accuracy:   0.572

Perceptron
________________________________________________________________________________
Training: 
Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=50, n_jobs=1, penalty=None, random_state=0, shuffle=True,
      verbose=0, warm_start=False)
accuracy:   0.578

Passive-Aggressive
________________________________________________________________________________
Training: 
PassiveAggressiveClassifier(C=1.0, class_weight=None, fit_intercept=True,
              loss='hinge', n_iter=50, n_jobs=1, random_state=None,
              shuffle=True, verbose=0, warm_start=False)
accuracy:   0.556

kNN
__________________________________________________________________

/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/svm/classes.py:199: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)


accuracy:   0.596

________________________________________________________________________________
Training: 
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=50, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
accuracy:   0.563

L1 penalty
________________________________________________________________________________
Training: 
LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='l2', max_iter=1000, multi_class='ovr',
     penalty='l1', random_state=None, tol=0.001, verbose=0)


/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/svm/classes.py:199: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)


accuracy:   0.587

________________________________________________________________________________
Training: 
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=50, n_jobs=1,
       penalty='l1', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
accuracy:   0.562

Elastic-Net penalty
________________________________________________________________________________
Training: 
SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=50, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
accuracy:   0.552

NearestCentroid (aka Rocchio classifier)
________________________________________________________________________________
Training: 
Near

In [130]:
forest = RandomForestClassifier(n_estimators= 100)

In [131]:
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [132]:
y_pred = forest.predict(X_test)

In [44]:
from sklearn.metrics import accuracy_score

In [133]:
accuracy_score(y_test, y_pred)

0.63879485443466488

In [ ]:
#с дефолтом было 624
# 100 - 6435